In [4]:
from fundata.client import init_api_client
from fundata.dota2.raw.raw import *
from fundata.dota2.league.player import *
from fundata.dota2.league.team import *

init_api_client()
    

    
res=get_single_player(318371533)
print("2.查询选手信息")
print(res)

2.查询选手信息
{'retcode': 200, 'data': {'id': 318371533, 'real_name': 'Skill lay???', 'name': 'Skill lay???', 'nation': 'CN', 'position': 1}}


In [ ]:
from mysql.sqlConnect import sqlConnection,sqlDisconnection,sqlInsert,createTable
from mysql.dataHandler import api_transfer_sql,sqlColumn
def testApiSql():
    ##从API拿数据
    init_api_client()
    res_a=get_single_player(318371533)
    
    ##写入MySQL
    sqlConnection()
    
    #建表
    createTable("player_test",res_a,True)
    
    #将API返回结果，转化成参数
    param=api_transfer_sql(res_a)
    #插入数据
    str=sqlColumn(res_a)
    sql="insert into player_test("+str["key"]+") values ("+str["s"]+")"
    print (sql)
    row=sqlInsert(sql,param)
    print("插入数据")
    print(row)
    
    #关闭数据库
    sqlDisconnection()